[Reference](https://medium.com/@kevinkoech265/file-uploads-and-downloads-in-fastapi-a-comprehensive-guide-06e0b18bb245)

```
cd path/to/your/project/directory
python -m venv venv_name

# macOS or Linux
source venv_name/bin/activate
# Windows
venv_name\Scripts\activate
(myenv) user@your-computer:~/path/to/your/project/directory$
```

In [1]:
pip install fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
pip install "uvicorn[standard]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.


In [3]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def root():
    return {"message": "Hello World"}

```
uvicorn main:app --reload
```

In [4]:
import shutil

from fastapi import FastAPI, UploadFile, File

app = FastAPI()


@app.post('/files')
def get_file(file: bytes = File(...)):
    content = file.decode('utf-8')
    lines = content.split('\n')
    return {"content": lines}

In [5]:
pip install python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 792.9 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.


In [6]:
@app.post('/upload')
def upload_file(uploaded_file: UploadFile = File(...)):
    path = f"files/{uploaded_file.filename}"
    with open(path, 'w+b') as file:
        shutil.copyfileobj(uploaded_file.file, file)

    return {
        'file': uploaded_file.filename,
        'content': uploaded_file.content_type,
        'path': path,
    }

In [7]:
from fastapi.staticfiles import StaticFiles
app.mount('/files', StaticFiles(directory='files'),'files')

In [8]:
import shutil
import json
from fastapi import FastAPI, UploadFile, File, status
from fastapi.staticfiles import StaticFiles
from fastapi.exceptions import HTTPException

@app.post('/upload/file')
def upload_json(file:UploadFile = File(...)):
    if file.content_type != 'application/json':
        raise HTTPException(status_code=status.HTTP_400_BAD_REQUEST, detail="Wow, That's not allowed")
    data = json.loads(file.file.read())
    return {
        "filename":file.filename,
        "content":data
    }

In [9]:
import shutil
import json
import time

import yaml
from fastapi import FastAPI, UploadFile, File, status
from fastapi.responses import FileResponse
from fastapi.staticfiles import StaticFiles
from fastapi.exceptions import HTTPException
import os

app = FastAPI()


time_str = time.strftime('%Y-%m%d - %H%M%S')
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
UPLOAD_DIR = os.path.join(BASE_DIR, "uploads")


@app.post('/upload/download')
def upload_download(file:UploadFile = File(...)):
    if file.content_type != 'application/json':
        raise HTTPException(status_code=status.HTTP_400_BAD_REQUEST, detail="Wow, that's not allowed")
    data = json.loads(file.file.read())
    new_filename =  "{}_{}.yaml".format(os.path.splitext(file.filename)[0], time_str)
    SAVE_F = os.path.join(UPLOAD_DIR, new_filename)
    with open(SAVE_F, "w") as f:
        yaml.dump(data,f)
    return FileResponse(path=SAVE_F, media_type="application/octet-stream", filename=new_filename)